In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
import numpy as np
# from ChromaPalette.chroma_palette import *
from scib_metrics.benchmark import Benchmarker
import scib
import seaborn as sns
import os

import matplotlib.pyplot as plt
color_palette =  ['#E24A33', '#F39B7F','#17BECF', '#348ABD', 
                '#988ED5', '#777777', '#FBC15E', 
                '#8EBA42', '#FFA5FF', '#8491B4']
plt.rcParams['text.color'] = 'black'
sc.set_figure_params(dpi=300, facecolor='white')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [8, 8]


/mnt/sdc/dongjiayi/miniconda3/envs/python38/lib/python3.8/site-packages/scanpy/_settings.py:450: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [14]:
method = ['scPSM',  'Scanorama', ] # 
data = ['hPBMC', 'hPancreas', 'mCortex']
version = 'v2'
j = 2
i = 0

In [15]:
data_path = f"/mnt/sdc/dongjiayi/denoise/data/{data[j]}/{version}/" 
noisy_adata = sc.read_h5ad(data_path+'demo.h5ad')

print(method[i])
if method[i] == 'raw':
    denoise_adata = noisy_adata
elif method[i] in ['scPSM', 'SAVERX']:
    denoise_adata = sc.read_csv(data_path+f'denoise_data_{method[i]}.csv')
    denoise_adata.obs = noisy_adata.obs
    print(denoise_adata.obs)
else:
    denoise_adata = sc.read_h5ad(data_path+f'denoise_adata_{method[i]}.h5ad')
    if method[i] not in ["Beaconet", 'Scanorama', 'MNN']:
        denoise_adata.obs = noisy_adata.obs
    print(denoise_adata.obs)

print(denoise_adata.obs['batch'])
if method[i] in ['MAGIC', 'DeepImpute', 'SAVERX', 'scVI', 'Bis']:
    sc.pp.normalize_total(denoise_adata, target_sum=10000)
    sc.pp.log1p(denoise_adata)

if method[i] in ['MAGIC', 'DeepImpute', 'SAVERX', 'scVI', 'Bis']:
    sc.pp.combat(denoise_adata, key='batch')
    
sc.tl.pca(denoise_adata, svd_solver="arpack")

bm = Benchmarker(
    denoise_adata, 
    batch_key='batch',
    label_key='cellstate',
    embedding_obsm_keys=['X_pca'],
    n_jobs=-1,
    )
bm.benchmark()
    
df = bm.get_results(min_max_scale=False)

print(df)

scPSM
                                                                                                 NAME  \
Cortex1_Smart_seq2.M2_expt2_1                                           Cortex1_Smart_seq2.M2_expt2_1   
Cortex1_Smart_seq2.M2_expt2_100                                       Cortex1_Smart_seq2.M2_expt2_100   
Cortex1_Smart_seq2.M2_expt2_101                                       Cortex1_Smart_seq2.M2_expt2_101   
Cortex1_Smart_seq2.M2_expt2_103                                       Cortex1_Smart_seq2.M2_expt2_103   
Cortex1_Smart_seq2.M2_expt2_104                                       Cortex1_Smart_seq2.M2_expt2_104   
...                                                                                               ...   
Cortex2_sci_RNA_seqCCKVLANXX.052318-SciSeq-p3_H...  Cortex2_sci_RNA_seqCCKVLANXX.052318-SciSeq-p3_...   
Cortex2_sci_RNA_seqCCKVLANXX.052318-SciSeq-p3_H...  Cortex2_sci_RNA_seqCCKVLANXX.052318-SciSeq-p3_...   
Cortex2_sci_RNA_seqCCKVLANXX.052318-SciSeq-p3_H..

Embeddings:   0%|          | 0/1 [00:40<?, ?it/s]


ValueError: Each cell must have the same number of neighbors.